Predicciones IRNI

Instrucciones del proyecto
Descarga y prepara los datos. Explica el procedimiento.

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import numpy as np


# Cargar los datos en DataFrames
df = pd.read_csv('ingresos.csv', sep=';')

# Mostrar las primeras filas de cada conjunto de datos
display(df.head())
display(df.columns)


,Mes,Ingreso,Costo Bienes,Utilidad Bruta,Gastos operativos,IVA Retenido,ISR Retenido,Utilidad Neta,ISR GENERADO,Utilidad Retirada
0,ene-25,"$114.647,00","$-106.100,00","$8.547,00","$-59.420,00","$-9.171,76","$-1.146,47","$-61.191,23","$-21.416,93","$10.000,00"
1,feb-25,"$128.579,00","$-38.200,00","$90.379,00","$-62.878,00","$-10.286,32","$-1.285,79","$15.928,89","$5.575,11","$9.000,00"
2,mar-25,"$125.142,42","$-42.750,00","$82.392,42","$-56.631,00","$-10.011,39","$-1.251,42","$14.498,60","$5.074,51","$10.000,00"
3,abr-25,"$117.645,00","$-74.600,00","$43.045,00","$-54.870,00","$-9.411,60","$-1.176,45","$-22.413,05","$-7.844,57","$8.000,00"
4,ene-24,"$48.654,00","$56.700,00","$-8.046,00","$22.638,48",NaN,NaN,NaN,NaN,NaN


Index(['Mes', 'Ingreso', 'Costo Bienes', 'Utilidad Bruta', 'Gastos operativos',
       'IVA Retenido', 'ISR Retenido', 'Utilidad Neta', 'ISR GENERADO',
       'Utilidad Retirada'],
      dtype='object')

In [ ]:
def clean_and_convert(df):
    columns = ['Mes', 'Ingreso', 'Costo Bienes', 'Utilidad Bruta', 'Gastos operativos', 'IVA Retenido', 'ISR Retenido', 'Utilidad Neta', 'ISR GENERADO', 'Utilidad Retirada']
    
    # Clean and convert numeric columns
    for col in df.columns[1:]:  # Skip the first column (Mes)
        # Remove spaces, currency symbols, and commas
        df[col] = df[col].str.strip()
        df[col] = df[col].str.replace('$', '')
        df[col] = df[col].str.replace(',', '')
        df[col] = df[col].str.replace('.', '')
        
        # Convert to float, handling any errors
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

# Apply the cleaning function
df = clean_and_convert(df)

# Verify the conversion
print(df.dtypes)
print("\nFirst few rows after conversion:")
# Remove rows where 'Utilidad Neta' is NaN
df = df.dropna(subset=['Utilidad Bruta'])
print(df.head())
display(df)

Mes                   object
Ingreso              float64
Costo Bienes         float64
Utilidad Bruta       float64
Gastos operativos    float64
IVA Retenido         float64
ISR Retenido         float64
Utilidad Neta        float64
ISR GENERADO         float64
Utilidad Retirada    float64
dtype: object

First few rows after conversion:
      Mes    Ingreso  Costo Bienes  Utilidad Bruta  Gastos operativos  \
0  ene-25  114.64700       -106.10         8.54700          -59.42000   
1  feb-25  128.57900        -38.20        90.37900          -62.87800   
2  mar-25  125.14242        -42.75        82.39242          -56.63100   
3  abr-25  117.64500        -74.60        43.04500          -54.87000   
4  ene-24   48.65400         56.70        -8.04600           22.63848   

   IVA Retenido  ISR Retenido  Utilidad Neta  ISR GENERADO  Utilidad Retirada  
0      -9.17176      -1.14647      -61.19123     -21.41693               10.0  
1     -10.28632      -1.28579       15.92889       5.57511     

,Mes,Ingreso,Costo Bienes,Utilidad Bruta,Gastos operativos,IVA Retenido,ISR Retenido,Utilidad Neta,ISR GENERADO,Utilidad Retirada
0,ene-25,114.64700,-106.100,8.54700,-5.942000e+01,-9.17176,-1.14647,-61.19123,-21.41693,10.0
1,feb-25,128.57900,-38.200,90.37900,-6.287800e+01,-10.28632,-1.28579,15.92889,5.57511,9.0
2,mar-25,125.14242,-42.750,82.39242,-5.663100e+01,-10.01139,-1.25142,14.49860,5.07451,10.0
3,abr-25,117.64500,-74.600,43.04500,-5.487000e+01,-9.41160,-1.17645,-22.41305,-7.84457,8.0
4,ene-24,48.65400,56.700,-8.04600,2.263848e+01,NaN,NaN,NaN,NaN,NaN
5,feb-24,63.03693,40.600,22.43693,2.456443e+01,NaN,NaN,NaN,NaN,NaN
6,mar-24,72.64568,54.620,18.02568,2.471748e+01,NaN,NaN,NaN,NaN,NaN
7,abr-24,90.87392,32.900,57.97392,2.270487e+01,NaN,NaN,NaN,NaN,NaN
8,may-24,91.70276,29.320,62.38276,2.160433e+01,NaN,NaN,NaN,NaN,NaN
9,jun-24,81.12223,65.450,15.67223,1.973467e+01,NaN,NaN,NaN,NaN,NaN


**2. Explorar la estructura de los datos
Verificamos las columnas y tipos de datos con .info()**

**Entrena y prueba el modelo para cada región en geo_data_0.csv:**

1.Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

2.Entrena un modelo de regresión lineal.

3.Entrena un modelo de regresión lineal.

4.Hace predicciones en el conjunto de validación.

5.Calcula y muestra el RMSE y el volumen medio de reservas predicho.

In [58]:

# Now modify the train_and_evaluate function to use a more appropriate target:
def train_and_evaluate(df, region_name):
    print(f"\nRegión {region_name}")
    
    # Separar características y variable objetivo
    # Use Mes as a feature instead of target
    features = df[['Ingreso', 'Costo Bienes', 'Gastos operativos']]
    target = df['Utilidad Bruta']  # Predicting net utility instead of Mes

    # Continue with the rest of your function...
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42
    )

    # Rest of your function remains the same...
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    # Update the print statements to reflect the new target
    print(f"Predicciones: {predictions[:5]}")  
    print(f"Valores reales: {target_valid[:5].values}")  
    mse = mean_squared_error(target_valid, predictions)
    rmse = np.sqrt(mse)
    print(f"RMSE del modelo: {rmse:.2f}")
    print(f"Utilidad Neta promedio predicha: {predictions.mean():.2f}")
    
    return model, predictions, target_valid

# Apply to region
model_0, predictions_0, target_valid_0 = train_and_evaluate(df, "Amazon")


Región Amazon
Predicciones: [ 37.95985668 857.05301841  48.47667844  51.0915878   71.27254567]
Valores reales: [15.67223 51.16772 62.38276 50.913    8.547  ]
RMSE del modelo: 305.70
Utilidad Neta promedio predicha: 169.43


✅**Expliacion**


 Calcular el error cuadrático medio (MSE)

mean_squared_error(y_real, y_pred) calcula el error cuadrático medio (MSE), que mide qué tan lejos están las predicciones de los valores reales.

2️⃣ Obtener la raíz del error cuadrático medio (RMSE)

La raíz cuadrada del MSE nos da el error cuadrático medio raíz (RMSE).


3️⃣ Mostrar resultados

rmse:.2f formatea el número con 2 decimales.
predictions.mean() nos da el promedio de reservas predichas, lo que nos ayuda a entender si el modelo sobreestima o subestima el valor real.



**3 Preparate para el calculo de ganancias**


**Paso 3.1: Almacenar valores clave**

Definimos las variables necesarias:

In [ ]:
# Costo total de inversión en millones de dólares
total_budget = 100_000_000  

# Número de pozos seleccionados
num_selected_wells = 200  

# Precio por barril en dólares
price_per_barrel = 4.5  

# Ingreso por cada unidad (miles de barriles)
revenue_per_unit = price_per_barrel * 1000  

# Punto de equilibrio por pozo
break_even_units = total_budget / (num_selected_wells * revenue_per_unit) 

print(f"Cada pozo debe producir al menos {break_even_units:.1f} unidades (miles de barriles) para evitar pérdidas.")


**Paso 3.2: Comparación con la cantidad media de reservas**

Cada pozo debe generar 111.1 unidades para evitar pérdidas.
El volumen medio de reservas en la región 0 nos dirá si en promedio cada pozo está por encima o por debajo de este umbral.


Si el valor medio de reservas en la región es menor a 111.1 unidades, perforar al azar sería una estrategia arriesgada. En este caso, la clave será seleccionar solo los mejores pozos.

In [ ]:
# Calcular la media de reservas en cada región
mean_reserves_0 = df_0['product'].mean()
mean_reserves_1 = df_1['product'].mean()
mean_reserves_2 = df_2['product'].mean()

print(f"Media de reservas - Región 0: {mean_reserves_0:.1f} unidades")
print(f"Media de reservas - Región 1: {mean_reserves_1:.1f} unidades")
print(f"Media de reservas - Región 2: {mean_reserves_2:.1f} unidades")


✅**3.3 Conclusiones sobre la preparación**✅

El pozo promedio no es suficiente para ninguna region

**Paso 4.1: Seleccionar los 200 pozos con las predicciones más altas**

Ordenamos las predicciones y seleccionamos los 200 pozos con el mayor valor predicho.

In [ ]:


num_selected_wells = 200  # Número de pozos a seleccionar

# Dividir los datos en entrenamiento (75%) y validación (25%) para cada región
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    df_0[['f0', 'f1', 'f2']], df_0['product'], test_size=0.25, random_state=42
)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    df_1[['f0', 'f1', 'f2']], df_1['product'], test_size=0.25, random_state=42
)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    df_2[['f0', 'f1', 'f2']], df_2['product'], test_size=0.25, random_state=42
)

# Entrenar modelos de regresión lineal para cada región
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)

# Función para seleccionar los 200 mejores pozos por predicción
def select_top_wells(features, target, model):
    predictions = model.predict(features)
    top_indices = np.argsort(predictions)[-num_selected_wells:]  # Seleccionar los 200 mejores
    return target.iloc[top_indices], predictions[top_indices]

# Seleccionar los mejores pozos en cada región
top_target_0, top_predictions_0 = select_top_wells(features_valid_0, target_valid_0, model_0)
top_target_1, top_predictions_1 = select_top_wells(features_valid_1, target_valid_1, model_1)
top_target_2, top_predictions_2 = select_top_wells(features_valid_2, target_valid_2, model_2)

# Mostrar información de los 200 mejores pozos por región
print(f"Volumen medio de reservas en los mejores 200 pozos - Región 0: {top_target_0.mean():.2f}")
print(f"Volumen medio de reservas en los mejores 200 pozos - Región 1: {top_target_1.mean():.2f}")
print(f"Volumen medio de reservas en los mejores 200 pozos - Región 2: {top_target_2.mean():.2f}")


**Paso 4.2: Calcular el volumen objetivo de reservas**

Sumamos las reservas reales (actual) de los 200 mejores pozos para ver cuántas reservas reales tienen.

In [ ]:
# Calcular el volumen total de reservas en los 200 mejores pozos de cada región
total_reserves_0 = top_target_0.sum()
total_reserves_1 = top_target_1.sum()
total_reserves_2 = top_target_2.sum()

# Mostrar los resultados
print(f"Volumen total de reservas en los 200 mejores pozos - Región 0: {total_reserves_0:.2f}")
print(f"Volumen total de reservas en los 200 mejores pozos - Región 1: {total_reserves_1:.2f}")
print(f"Volumen total de reservas en los 200 mejores pozos - Región 2: {total_reserves_2:.2f}")


**Paso 4.3: Calcular la ganancia potencial**


Utilizamos la fórmula:

Ganancia=Total de reservas×4500−100,000,000


In [ ]:
# Definir constantes
REVENUE_PER_UNIT = 4500  # Ingreso por unidad de reserva
INVESTMENT = 100_000_000  # Inversión total en dólares

# Calcular la ganancia potencial en cada región
profit_0 = total_reserves_0 * REVENUE_PER_UNIT - INVESTMENT
profit_1 = total_reserves_1 * REVENUE_PER_UNIT - INVESTMENT
profit_2 = total_reserves_2 * REVENUE_PER_UNIT - INVESTMENT

# Mostrar los resultados
print(f"Ganancia potencial - Región 0: ${profit_0:,.2f}")
print(f"Ganancia potencial - Región 1: ${profit_1:,.2f}")
print(f"Ganancia potencial - Región 2: ${profit_2:,.2f}")


✅**Conclusión**✅

Volumen objetivo de reservas:

Al sumar las reservas reales de los 200 mejores pozos en cada región, observamos diferencias significativas en la capacidad productiva de cada zona.
Ganancia potencial:

la region 0 obtuvo las mejores ganancias con $33M, seguido de la Region 2 con casi $26M y por ultimo la Region 1 con $24M

**Paso 5.1: Aplicar Bootstrapping**

Generamos 1000 muestras aleatorias con reemplazo y calculamos la ganancia para cada una.

In [ ]:

# Definir la cantidad de simulaciones
num_bootstrap_samples = 1000

# Función para calcular la ganancia con bootstrapping
def bootstrap_profit(data_df, actual_reserves_col, predicted_reserves_col, 
                     num_total_wells_to_sample=500, num_selected_wells=200):
    """
    Calcula la distribución de la ganancia utilizando la técnica de bootstrapping.

    Args:
        data_df (pd.DataFrame): DataFrame que contiene los datos de los pozos.
                               Debe incluir las columnas de reservas reales y predichas.
                               Se espera que este DataFrame contenga los 25,000 pozos de validación.
        actual_reserves_col (str): Nombre de la columna con las reservas reales.
        predicted_reserves_col (str): Nombre de la columna con las reservas predichas.
        num_total_wells_to_sample (int): Número de pozos a muestrear con reemplazo en cada iteración (e.g., 500).
        num_selected_wells (int): Número de los mejores pozos a seleccionar basados en la predicción (e.g., 200).

    Returns:
        np.array: Un array con las ganancias calculadas para cada muestra de bootstrapping.
    """
    profits = []
    
    # Costo de desarrollo de 200 pozos (100 millones / 200 pozos = 500,000 por pozo)
    # Asumimos que el costo total es de 100_000_000 para los num_selected_wells
    cost_per_well_development = 100_000_000 / num_selected_wells 
    total_development_cost = 100_000_000 # Costo total fijo para los 200 pozos seleccionados

    # Precio por unidad de reserva (ej. 4500 por mil barriles, o $4.5 por barril si las reservas están en barriles)
    price_per_unit_reserve = 4500 

    for _ in range(num_bootstrap_samples):
        # 1. Seleccionar una muestra aleatoria de 'num_total_wells_to_sample' pozos con reemplazo
        #    del conjunto de validación (los 25,000 pozos).
        sample_of_500 = data_df.sample(n=num_total_wells_to_sample, replace=True)
        
        # 2. De esta muestra de 500, seleccionar los 'num_selected_wells' (200) pozos
        #    con las reservas PREDICHAS más altas.
        top_sample_by_prediction = sample_of_500.nlargest(num_selected_wells, columns=predicted_reserves_col)
        
        # 3. Sumar las reservas REALES de estos 200 pozos seleccionados.
        total_actual_reserves_from_selected = top_sample_by_prediction[actual_reserves_col].sum()
        
        # 4. Aplicar la fórmula de ganancia.
        #    Ganancia = (Volumen total de reservas reales * Precio por unidad) - Costo total de desarrollo
        profit = total_actual_reserves_from_selected * price_per_unit_reserve - total_development_cost
        profits.append(profit)
        
    return np.array(profits)

# --- Ejemplo de cómo usarlo ---
# Supongamos que tienes DataFrames para cada región:
# validation_data_0, validation_data_1, validation_data_2
# Y que cada DataFrame tiene columnas llamadas 'actual_reserves' y 'predicted_reserves'.

# Ejemplo de creación de DataFrames de validación (simulados para demostración)
# En tu caso, estos serían los DataFrames con los 25,000 pozos de validación.
np.random.seed(42) # para reproducibilidad
data_size = 25000 

validation_data_0 = pd.DataFrame({
    'actual_reserves': np.random.rand(data_size) * 200 + 50, # Reservas reales entre 50 y 250
    'predicted_reserves': np.random.rand(data_size) * 200 + np.random.normal(0, 20, data_size) + 50 # Predicciones con algo de ruido
})
validation_data_1 = pd.DataFrame({
    'actual_reserves': np.random.rand(data_size) * 220 + 40,
    'predicted_reserves': np.random.rand(data_size) * 220 + np.random.normal(0, 25, data_size) + 40
})
validation_data_2 = pd.DataFrame({
    'actual_reserves': np.random.rand(data_size) * 180 + 60,
    'predicted_reserves': np.random.rand(data_size) * 180 + np.random.normal(0, 15, data_size) + 60
})

# Nombres de las columnas (asegúrate que coincidan con tus DataFrames)
actual_col = 'actual_reserves'
predicted_col = 'predicted_reserves'

# Aplicar bootstrapping a cada región
# Se asume que 'validation_data_0', 'validation_data_1', 'validation_data_2'
# son los DataFrames con los 25,000 pozos de validación para cada región.
bootstrap_profits_0 = bootstrap_profit(validation_data_0, actual_col, predicted_col)
bootstrap_profits_1 = bootstrap_profit(validation_data_1, actual_col, predicted_col)
bootstrap_profits_2 = bootstrap_profit(validation_data_2, actual_col, predicted_col)

# Mostrar un resumen de las ganancias obtenidas
print(f"Media de ganancia - Región 0: ${bootstrap_profits_0.mean():,.2f}")
print(f"Desviación estándar de ganancia - Región 0: ${bootstrap_profits_0.std():,.2f}")
print(f"Intervalo de confianza del 95% - Región 0: (${np.percentile(bootstrap_profits_0, 2.5):,.2f} - ${np.percentile(bootstrap_profits_0, 97.5):,.2f})")
print(f"Riesgo de pérdida (porcentaje de veces que la ganancia < 0) - Región 0: {np.mean(bootstrap_profits_0 < 0) * 100:.2f}%\n")

print(f"Media de ganancia - Región 1: ${bootstrap_profits_1.mean():,.2f}")
print(f"Desviación estándar de ganancia - Región 1: ${bootstrap_profits_1.std():,.2f}")
print(f"Intervalo de confianza del 95% - Región 1: (${np.percentile(bootstrap_profits_1, 2.5):,.2f} - ${np.percentile(bootstrap_profits_1, 97.5):,.2f})")
print(f"Riesgo de pérdida - Región 1: {np.mean(bootstrap_profits_1 < 0) * 100:.2f}%\n")

print(f"Media de ganancia - Región 2: ${bootstrap_profits_2.mean():,.2f}")
print(f"Desviación estándar de ganancia - Región 2: ${bootstrap_profits_2.std():,.2f}")
print(f"Intervalo de confianza del 95% - Región 2: (${np.percentile(bootstrap_profits_2, 2.5):,.2f} - ${np.percentile(bootstrap_profits_2, 97.5):,.2f})")
print(f"Riesgo de pérdida - Región 2: {np.mean(bootstrap_profits_2 < 0) * 100:.2f}%")


**Paso 5.2: Calcular estadísticas clave**


Extraemos el beneficio promedio, el intervalo de confianza del 95%, y el riesgo de pérdida (casos donde la ganancia es negativa).

In [ ]:
# Función para calcular estadísticas clave
def calculate_statistics(profits):
    mean_profit = profits.mean()  # Beneficio promedio
    lower_bound = np.percentile(profits, 2.5)  # Límite inferior del intervalo de confianza del 95%
    upper_bound = np.percentile(profits, 97.5)  # Límite superior del intervalo de confianza del 95%
    risk_of_loss = (profits < 0).mean()  # Proporción de muestras con pérdida (ganancia negativa)

    return mean_profit, lower_bound, upper_bound, risk_of_loss

# Calcular estadísticas para cada región
stats_0 = calculate_statistics(bootstrap_profits_0)
stats_1 = calculate_statistics(bootstrap_profits_1)
stats_2 = calculate_statistics(bootstrap_profits_2)

# Mostrar resultados
print(f"Región 0: Media de ganancia: ${stats_0[0]:,.2f}, IC 95%: (${stats_0[1]:,.2f}, ${stats_0[2]:,.2f}), Riesgo de pérdida: {stats_0[3]:.2%}")
print(f"Región 1: Media de ganancia: ${stats_1[0]:,.2f}, IC 95%: (${stats_1[1]:,.2f}, ${stats_1[2]:,.2f}), Riesgo de pérdida: {stats_1[3]:.2%}")
print(f"Región 2: Media de ganancia: ${stats_2[0]:,.2f}, IC 95%: (${stats_2[1]:,.2f}, ${stats_2[2]:,.2f}), Riesgo de pérdida: {stats_2[3]:.2%}")


**Paso 5.3: Conclusión**

Conclusión del Análisis con Bootstrapping
Tras aplicar bootstrapping con 1000 simulaciones, obtuvimos el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdida para cada región.

Resultados clave:
La región con la mayor ganancia promedio es la mejor candidata para la inversión.
Si alguna región tiene un riesgo de pérdida superior al 2.5%-5%, es una opción más riesgosa.
El intervalo de confianza del 95% nos indica el rango en el que es probable que se encuentren las ganancias reales.
Decisión Final:
Si una región tiene un beneficio alto y un bajo riesgo de pérdida, es la mejor opción para invertir. En cambio, si los beneficios son similares entre regiones, pero una tiene menos riesgo, esa sería la más segura.

Con estos resultados, podemos tomar una decisión basada en datos para maximizar la rentabilidad y minimizar riesgos. 

**Propuesta de Región para el Desarrollo de Pozos Petrolíferos**

Con base en los análisis realizados, la región óptima para el desarrollo de pozos petrolíferos es la que mostró:

✅ Mayor ganancia promedio en el bootstrapping.
✅ Menor riesgo de pérdida (probabilidad de obtener ganancias negativas).
✅ Intervalo de confianza del 95% más alto, lo que sugiere estabilidad en las ganancias.

¿Coincide con la elección del paso 4.3?
Si la región seleccionada en el paso 4.3 (basado en la predicción de los 200 mejores pozos) es la misma que resultó más rentable en el análisis de riesgos del bootstrapping, entonces sí coincide. Esto validaría que nuestras predicciones eran acertadas.

Sin embargo, si la región con más reservas en el paso 4.3 tiene un alto riesgo de pérdida en el bootstrapping, podría ser mejor reconsiderar la inversión y optar por una región más estable.

Justificación Final
La región seleccionada maximiza el beneficio esperado mientras minimiza el riesgo. Al tomar en cuenta tanto las predicciones iniciales como el análisis de simulaciones, podemos hacer una inversión más informada y estratégica